In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer

In [51]:
df_ = pd.read_csv('hackupc_provincies.csv')

In [52]:
df = df_

,price,square_meters,bedrooms,bathrooms,num_images,image_data.r1f6.property,baseboard_radiator,central_ac,dishwasher,double_oven,...,id_prov_41.0,id_prov_42.0,id_prov_43.0,id_prov_44.0,id_prov_45.0,id_prov_46.0,id_prov_47.0,id_prov_48.0,id_prov_49.0,id_prov_50.0
count,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,...,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000
mean,173170.552586,118.643320,3.017359,1.713722,12.901866,3.858070,0.000417,0.004086,0.122544,0.003328,...,0.008064,0.032888,0.016250,0.018259,0.009304,0.028042,0.014083,0.011150,0.002767,0.016385
std,131901.415833,61.174251,1.014441,0.768280,12.424585,0.617113,0.020417,0.063788,0.327913,0.057590,...,0.089438,0.178344,0.126434,0.133885,0.096007,0.165092,0.117832,0.105001,0.052529,0.126950
min,15000.000000,46.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,85000.000000,78.000000,2.000000,1.000000,1.000000,3.500000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,135000.000000,100.000000,3.000000,2.000000,11.000000,3.900000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,217000.000000,137.000000,3.114558,2.000000,20.000000,4.300000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,845000.000000,365.000000,19.000000,15.000000,150.000000,6.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [53]:
# drop columnes que no ens interessen: 
columns_to_drop = ['Unnamed: 0', 'Unnamed: 0.1', "property_id", "summary", "region", "images", "beach_view"] 
df = df.drop(columns_to_drop, axis=1).reset_index(drop= True)
df['id_prov'] = df['id_prov'].astype('category')

label_encoder = LabelEncoder()
df['property_type_numeric'] = label_encoder.fit_transform(df['property_type'])

DF = df.drop(columns=['property_type']).reset_index(drop= True)

In [54]:
DF = DF.dropna(subset=['id_prov']).reset_index(drop= True)
DF = pd.get_dummies(DF, columns=['id_prov'])
DF = DF.drop('id_prov_51.0', axis=1)


# per millorar l'accuracy traiem outliers
Qp = DF['price'].quantile(0.97)
DF = DF.drop(DF[(DF['price'] > Qp)].index)

Qsm = DF['square_meters'].quantile(0.05)
DF = DF.drop(DF[(DF['square_meters'] <= Qsm)].index)
Qsm_ = DF['square_meters'].quantile(0.97)
DF = DF.drop(DF[(DF['square_meters'] > Qsm_)].index)

In [41]:
DF.describe()

,price,square_meters,bedrooms,bathrooms,num_images,image_data.r1f6.property,baseboard_radiator,central_ac,dishwasher,double_oven,...,id_prov_41.0,id_prov_42.0,id_prov_43.0,id_prov_44.0,id_prov_45.0,id_prov_46.0,id_prov_47.0,id_prov_48.0,id_prov_49.0,id_prov_50.0
count,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,...,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000,354902.000000
mean,173170.552586,118.643320,3.017359,1.713722,12.901866,3.858070,0.000417,0.004086,0.122544,0.003328,...,0.008064,0.032888,0.016250,0.018259,0.009304,0.028042,0.014083,0.011150,0.002767,0.016385
std,131901.415833,61.174251,1.014441,0.768280,12.424585,0.617113,0.020417,0.063788,0.327913,0.057590,...,0.089438,0.178344,0.126434,0.133885,0.096007,0.165092,0.117832,0.105001,0.052529,0.126950
min,15000.000000,46.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,85000.000000,78.000000,2.000000,1.000000,1.000000,3.500000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,135000.000000,100.000000,3.000000,2.000000,11.000000,3.900000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,217000.000000,137.000000,3.114558,2.000000,20.000000,4.300000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,845000.000000,365.000000,19.000000,15.000000,150.000000,6.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [30]:
def rmspe(y_true, y_pred):
    # Calcular el error porcentual
    percentage_error = (y_true - y_pred) / y_true
    # Ignorar divisiones por cero y elementos infinitos
    percentage_error = np.where(np.isfinite(percentage_error), percentage_error, 0)
    # Calcular el RMSPE
    rmspe = np.sqrt(np.mean(percentage_error**2))
    return rmspe

In [72]:
DF = DF.drop('logprice', axis = 1)

X = DF.drop('price', axis=1)
y = DF['price']

In [84]:
X.to_csv("final_house_dataframe.csv")

square_meters               float64
bedrooms                    float64
bathrooms                   float64
num_images                    int64
image_data.r1f6.property    float64
                             ...   
id_prov_46.0                  uint8
id_prov_47.0                  uint8
id_prov_48.0                  uint8
id_prov_49.0                  uint8
id_prov_50.0                  uint8
Length: 171, dtype: object

In [17]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

We have tried many models and it turns out the best one was the one with the default parameters.

In [91]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, min_samples_leaf=5, n_estimators=400)

In [92]:
y_pred = model.predict(X_test)

In [93]:
mse = mean_squared_error(y_test, y_pred)

Linear Regression and Bayesian Inference where also considered.

In [120]:
ridge_cross_val_metrics = pd.DataFrame(columns=['mean MSE', 'mean norm_MSE', 'mean R2'])
lambdas = [0, 0.01, 0.1, 0.5, 1]

# We calculate the cross-validation metrics for each lambda 
for lambda_val in lambdas:
    kf = KFold(n_splits=5)
    i=1
    cv_mse = []
    
    cv_nmse = []
    cv_r2 = []
    j = 1
    # We compute the metrics for each fold and then perform the mean.
    for train_index, test_index in kf.split(X_train):
        x_train_fold = X_train.values[train_index]
        y_train_fold = y_train.values[train_index]
        x_test_fold = X_train.values[test_index,:]
        y_test_fold = y_train.values[test_index]

        lr = Ridge(alpha=lambda_val)
        lr.fit(x_train_fold,y_train_fold)
        y_pred_fold = lr.predict(x_test_fold)
        fold_mse =mean_squared_error(y_test_fold, y_pred_fold)
        fold_nmse =  1-r2_score(y_test_fold, y_pred_fold)
        fold_r2 = r2_score(y_test_fold, y_pred_fold)
        print(y_test_fold.shape)
        cv_mse.append(fold_mse)
        cv_nmse.append(fold_nmse)
        cv_r2.append(fold_r2)

    
    ridge_cross_val_metrics.loc['Lambda={}'.format(lambda_val),:] = [np.mean(cv_mse),np.mean(cv_nmse),np.mean(cv_r2)]
    
ridge_cross_val_metrics.sort_values(by='mean R2',ascending=False)

(59762,)
(59762,)
(59761,)
(59761,)
(59761,)
(59762,)
(59762,)
(59761,)
(59761,)
(59761,)
(59762,)
(59762,)
(59761,)
(59761,)
(59761,)
(59762,)
(59762,)
(59761,)
(59761,)
(59761,)
(59762,)
(59762,)
(59761,)
(59761,)
(59761,)


,mean MSE,mean norm_MSE,mean R2
Lambda=0,0.31667,0.485157,0.514843
Lambda=0.01,0.31667,0.485157,0.514843
Lambda=0.1,0.31667,0.485157,0.514843
Lambda=0.5,0.31667,0.485157,0.514843
Lambda=1,0.31667,0.485157,0.514843
